In [5]:
import numpy as np
import scipy.io
import tensorflow as tf
import matplotlib.pyplot as plt

In [82]:
# let's load the IRMAS dataset
IRMAS = scipy.io.loadmat('IRMAS_small.mat')
data = IRMAS['data']

X = data[0:3,0:-1:16]

X_q = np.round(X,decimals=2)

quants = np.unique(X_q)
nquants = quants.shape[0]

N,D = X_q.shape

X = np.zeros((N,D,nquants,1))

for i in xrange(N):
    for j in xrange(D):
        i_hot = np.where(X_q[i,j] == quants) 
        X[i,j,i_hot[0][0]] = 1
        
print X.shape        

(3, 2757, 55, 1)


In [92]:
lr = 0.01

X_tf = tf.placeholder("float", [None,D,nquants,1])
y_tf = tf.placeholder("float", [None,nquants])

C1D = 2
C1ch = 3
C2D = 1
C2ch = 4

W1 = tf.Variable(tf.truncated_normal([C1D,nquants,1,C1ch],stddev=0.001))
W2 = tf.concat([tf.truncated_normal([C2D,1,C1ch,C2ch],stddev=0.001), 
                tf.zeros((1,1,C1ch,C2ch)),
               tf.truncated_normal([C2D,1,C1ch,C2ch],stddev=0.001)],0)

C1_out = tf.nn.conv2d(X_tf, W1, strides=[1,1,nquants,1],padding = 'SAME')
C1_out = tf.nn.relu(C1_out)
C2_out = tf.nn.conv2d(C1_out,W2, strides = [1,2,1,C1ch],padding = 'SAME' )
C2_out = tf.nn.relu(C2_out)

C2_out_flat = tf.reshape(C2_out,[-1,(D/2)*C2ch])